## Import Packages

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold
import joblib

In [2]:
df = pd.read_csv('../data/interim/2_data_engineered.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   HR_before            19 non-null     float64
 1   HR_after             19 non-null     float64
 2   sp_total_distance    19 non-null     float64
 3   sp_average_distance  19 non-null     float64
 4   sp_median_distance   19 non-null     float64
 5   sp_total_angle       19 non-null     float64
 6   sp_average_angle     19 non-null     float64
 7   sp_median_angle      19 non-null     float64
 8   row_id               19 non-null     float64
 9   total_latency        19 non-null     float64
 10  mean_latency         19 non-null     float64
 11  median_latency       19 non-null     float64
 12  vo2_max              19 non-null     float64
dtypes: float64(13)
memory usage: 2.1 KB


In [4]:
df = df.dropna()

## Choose your Feature!

In [5]:
df = df[['HR_before','vo2_max','sp_median_distance','sp_median_angle','median_latency']]

## Classify the vo2max

In [6]:
# Create the new feature
df['fitness_status'] = df['vo2_max'].apply(lambda x: 1 if x >= 30 else 0)


In [7]:
df.fitness_status.value_counts()

fitness_status
0    19
Name: count, dtype: int64

## Train Test Split

In [8]:
# Assuming 'df' is your DataFrame and 'target_column' is the name of the target variable
X = df.drop(columns=['vo2_max'])  # Features (drop the target column)
y = df['vo2_max']                 # Target (the column you're predicting)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
X

,HR_before,sp_median_distance,sp_median_angle,median_latency,fitness_status
0,72.0,30.113397,0.056699,500.0,0
1,84.0,25.021036,0.090627,458.0,0
2,94.0,35.104838,0.060777,583.0,0
3,83.0,16.529260,0.033269,581.0,0
4,70.0,22.943240,0.057553,579.0,0
5,68.0,21.586166,0.056654,542.0,0
6,68.0,23.436273,0.052490,620.0,0
7,78.0,22.129910,0.042896,560.5,0
8,85.0,34.291648,0.065083,562.0,0
9,96.0,28.681657,0.075349,624.5,0


## Define Evaluation Metrics Function

In [10]:
def evaluate_model(model, model_name, X, y):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform 5-fold cross-validation
    cv_scores = cross_validate(model, X, y, cv=kfold, scoring=['neg_root_mean_squared_error', 'r2', 'neg_mean_absolute_percentage_error'])

    # Extracting individual metrics from the results
    rmse_scores = -cv_scores['test_neg_root_mean_squared_error']  # Negate to make positive
    r2_scores = cv_scores['test_r2']
    mape_scores = -cv_scores['test_neg_mean_absolute_percentage_error']  # Negate to make positive

    return [model_name, round(float(rmse_scores.mean()),5), round(float(r2_scores.mean()),5), round(float(mape_scores.mean())*100,5)]

## Modelling

### Linear Regression

In [11]:
# Initialize the Linear Regression model
lr = LinearRegression()

# Train the model on the training data
lr.fit(X_train, y_train)

# Make predictions on the test data
y_pred_lr = lr.predict(X_test)

lr_scores = evaluate_model(lr, 'Linear Regression', X, y)

NameError: name 'LinearRegression' is not defined

## Random Forest

In [10]:
rf = RandomForestRegressor(random_state=42)

# rf.fit(X_train, y_train)

# y_pred_lr = rf.predict(X_test)

rf_scores = evaluate_model(rf, 'Random Forest Regressor', X, y)

## XGBoost

In [11]:
# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(random_state=42)

xgb_scores = evaluate_model(xgb_model, 'XGBoost Model', X, y)

## Put into dataframe

In [12]:
scores = [lr_scores, rf_scores, xgb_scores]
scores = pd.DataFrame(scores, columns=['Model Name', 'RMSE', 'R2', 'MAPE'])
scores

,Model Name,RMSE,R2,MAPE
0,Linear Regression,2.03238,0.44474,5.85206
1,Random Forest Regressor,2.58816,0.13475,8.71408
2,XGBoost Model,3.08176,-0.14498,10.97197


## Save model into dataframe

Since the best model is linear regression, we will save linreg model

In [13]:
# Initialize the Linear Regression model
lr = LinearRegression()

# Train the model on the training data
lr.fit(X, y)

# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Assuming 'model' is your trained model
joblib.dump(lr, f'../models/linear_regression_{timestamp}.pkl')

['../models/linear_regression_20241029_182229.pkl']